In [1]:
#!pip3 install pandas 
#!pip install --upgrade pip

In [2]:
#!pip install --upgrade --user pip

In [3]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [4]:
df.loc[:,['Gold', 'Gold.1', 'Gold.2']].head()

,Gold,Gold.1,Gold.2
Afghanistan,0,0,0
Algeria,5,0,5
Argentina,18,0,18
Armenia,1,0,1
Australasia,3,0,3


In [5]:
df[df['Gold.2'] == max(df['Gold.2'])]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
United States,26,976,757,666,2399,22,96,102,84,282,48,1072,859,750,2681,USA


In [6]:
df[df['Gold.2'] == max(df['Gold.2'])].index[0]

'United States'

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [7]:
difGG1 = abs(df['Gold'] - df['Gold.1'])

In [8]:
difGG1.head()

Afghanistan     0
Algeria         5
Argentina      18
Armenia         1
Australasia     3
dtype: int64

In [9]:
max(difGG1)

880

In [10]:
df[df['Gold'] - df['Gold.1'] == max(difGG1)]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
United States,26,976,757,666,2399,22,96,102,84,282,48,1072,859,750,2681,USA


In [11]:
df[df['Gold'] - df['Gold.1'] == max(difGG1)].index[0]

'United States'

In [12]:
difGG1[difGG1 == max(difGG1)].index[0]

'United States'

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [13]:
difGG1TR = difGG1 / df['Gold.2']

In [14]:
max(difGG1TR.where(difGG1TR < 1).dropna())

0.9615384615384616

In [15]:
difGG1TR[difGG1TR == max(difGG1TR.where(difGG1TR < 1).dropna())].index[0]

'Bulgaria'

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created, with the country names as indices.

*This function should return a Series named `Points` of length 146*

In [16]:
points1 = df['Gold.2']*3 + df['Silver.2']*2 + df['Bronze.2']
p1 = pd.Series(points1)
p1.head()

Afghanistan      2
Algeria         27
Argentina      130
Armenia         16
Australasia     22
dtype: int64

In [17]:
len(points1)

146

In [18]:
def puntos():
    p2 = df['Gold.2']*3 + df['Silver.2']*2 + df['Bronze.2']
    Points = pd.Series(p2)
    return Points

In [19]:
puntos().head()

Afghanistan      2
Algeria         27
Argentina      130
Armenia         16
Australasia     22
dtype: int64

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [185]:
census_df = pd.read_csv('census.csv')
census_df

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861
5,50,3,6,1,9,Alabama,Blount County,57322,57322,57373,...,1.807375,-1.177622,-1.748766,-2.062535,-1.369970,1.859511,-0.848580,-1.402476,-1.577232,-0.884411
6,50,3,6,1,11,Alabama,Bullock County,10914,10915,10887,...,-30.953709,-5.180127,-1.130263,14.354290,-16.167247,-29.001673,-2.825524,1.507017,17.243790,-13.193961
7,50,3,6,1,13,Alabama,Butler County,20947,20946,20944,...,-14.032727,-11.684234,-5.655413,1.085428,-6.529805,-13.936612,-11.586865,-5.557058,1.184103,-6.430868
8,50,3,6,1,15,Alabama,Calhoun County,118572,118586,118437,...,-6.155670,-4.611706,-5.524649,-4.463211,-3.376322,-5.791579,-4.092677,-5.062836,-3.912834,-2.806406
9,50,3,6,1,17,Alabama,Chambers County,34215,34170,34098,...,-2.731639,3.849092,2.872721,-2.287222,1.349468,-1.821092,4.701181,3.781439,-1.290228,2.346901


### Ejercicios DEA

In [22]:
## ¿Cuántos países tienen Gold?
df[df['Gold']>0]['Gold'].count()

99

In [23]:
## ¿Cuántos países tienen Gold.1?
df[df['Gold.1']>0]['Gold.1'].count()

37

In [24]:
df[df['Gold.1']>0]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Australia,25,139,152,177,468,18,5,3,4,12,43,144,155,181,480,AUS
Austria,26,18,33,35,86,22,59,78,81,218,48,77,111,116,304,AUT
Belarus,5,12,24,39,75,6,6,4,5,15,11,18,28,44,90,BLR
Belgium,25,37,52,53,142,20,1,1,3,5,45,38,53,56,147,BEL
Bulgaria,19,51,85,78,214,19,1,2,3,6,38,52,87,81,220,BUL
Canada,25,59,99,121,279,22,62,56,52,170,47,121,155,173,449,CAN
China,9,201,146,126,473,10,12,22,19,53,19,213,168,145,526,CHN
Croatia,6,6,7,10,23,7,4,6,1,11,13,10,13,11,34,CRO
Czech Republic,5,14,15,15,44,6,7,9,8,24,11,21,24,23,68,CZE
Czechoslovakia,16,49,49,45,143,16,2,8,15,25,32,51,57,60,168,TCH


In [25]:
## ¿Cuántos países tienen Gold y Gold.1?
## Países que han ganado medalla de oro en Summer y Winter
len(df[(df['Gold']>0) & (df['Gold.1']>0)])

36

In [26]:
df[(df['Gold']>0) & (df['Gold.1']>0)].head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Australia,25,139,152,177,468,18,5,3,4,12,43,144,155,181,480,AUS
Austria,26,18,33,35,86,22,59,78,81,218,48,77,111,116,304,AUT
Belarus,5,12,24,39,75,6,6,4,5,15,11,18,28,44,90,BLR
Belgium,25,37,52,53,142,20,1,1,3,5,45,38,53,56,147,BEL
Bulgaria,19,51,85,78,214,19,1,2,3,6,38,52,87,81,220,BUL


In [27]:
##Países que han ganado medalla de oro en Summer y no en Winter
len(df[(df['Gold']>0)&(df['Gold.1']==0)])

63

In [28]:
df[(df['Gold']>0)&(df['Gold.1']==0)].head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ
Azerbaijan,5,6,5,15,26,5,0,0,0,0,10,6,5,15,26,AZE


In [29]:
## Países que han ganado medalla de oro en Winter y no en Summer
df[(df['Gold']==0)&(df['Gold.1']>0)]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Liechtenstein,16,0,0,0,0,18,2,2,5,9,34,2,2,5,9,LIE


In [30]:
len(df[(df['Gold']==0)&(df['Gold.1']>0)])

1

In [31]:
##Máximo ganador de oro Winter
df[df['Gold.1'] == max(df['Gold.1'])]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Norway,24,56,49,43,148,22,118,111,100,329,46,174,160,143,477,NOR


In [32]:
##¿Quién ha ido más veces a Winter?
df[df['# Winter'] == max(df['# Winter'])]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Austria,26,18,33,35,86,22,59,78,81,218,48,77,111,116,304,AUT
Canada,25,59,99,121,279,22,62,56,52,170,47,121,155,173,449,CAN
Finland,24,101,84,117,302,22,42,62,57,161,46,143,146,174,463,FIN
France,27,202,223,246,671,22,31,31,47,109,49,233,254,293,780,FRA
Great Britain,27,236,272,272,780,22,10,4,12,26,49,246,276,284,806,GBR
Hungary,25,167,144,165,476,22,0,2,4,6,47,167,146,169,482,HUN
Italy,26,198,166,185,549,22,37,34,43,114,48,235,200,228,663,ITA
Norway,24,56,49,43,148,22,118,111,100,329,46,174,160,143,477,NOR
Poland,20,64,82,125,271,22,6,7,7,20,42,70,89,132,291,POL
Sweden,26,143,164,176,483,22,50,40,54,144,48,193,204,230,627,SWE


In [33]:
##¿Quién ha ido más veces a Summer?
df[df['# Summer'] == max(df['# Summer'])]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
France,27,202,223,246,671,22,31,31,47,109,49,233,254,293,780,FRA
Great Britain,27,236,272,272,780,22,10,4,12,26,49,246,276,284,806,GBR
Greece,27,30,42,39,111,18,0,0,0,0,45,30,42,39,111,GRE
Switzerland,27,47,73,65,185,22,50,40,48,138,49,97,113,113,323,SUI


In [34]:
##Países que más han ido a Summer y Winter
df[(df['# Summer'] == max(df['# Summer']))&(df['# Winter'] == max(df['# Winter']))]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
France,27,202,223,246,671,22,31,31,47,109,49,233,254,293,780,FRA
Great Britain,27,236,272,272,780,22,10,4,12,26,49,246,276,284,806,GBR
Switzerland,27,47,73,65,185,22,50,40,48,138,49,97,113,113,323,SUI


In [35]:
##Máximo ganador de medallas
df[df['Combined total'] == max(df['Combined total'])]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
United States,26,976,757,666,2399,22,96,102,84,282,48,1072,859,750,2681,USA


In [36]:
##Máximo ganador de medallas en Summer
df[df['Total'] == max(df['Total'])]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
United States,26,976,757,666,2399,22,96,102,84,282,48,1072,859,750,2681,USA


In [37]:
##Máximo ganador de medallas en Winter
df[df['Total.1'] == max(df['Total.1'])]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Norway,24,56,49,43,148,22,118,111,100,329,46,174,160,143,477,NOR


In [38]:
##Países con 0 medallas totales
df[df['Combined total'] == 0]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID


In [39]:
##País con menos medallas totales
df[df['Combined total'] == min(df['Combined total'])]

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Bahrain,8,0,0,1,1,0,0,0,0,0,8,0,0,1,1,BRN
Barbados,11,0,0,1,1,0,0,0,0,0,11,0,0,1,1,BAR
Bermuda,17,0,0,1,1,7,0,0,0,0,24,0,0,1,1,BER
Botswana,9,0,1,0,1,0,0,0,0,0,9,0,1,0,1,BOT
Burundi,5,1,0,0,1,0,0,0,0,0,5,1,0,0,1,BDI
Ivory Coast,12,0,1,0,1,0,0,0,0,0,12,0,1,0,1,CIV
Cyprus,9,0,1,0,1,10,0,0,0,0,19,0,1,0,1,CYP
Djibouti,7,0,0,1,1,0,0,0,0,0,7,0,0,1,1,DJI
Eritrea,4,0,0,1,1,0,0,0,0,0,4,0,0,1,1,ERI
Gabon,9,0,1,0,1,0,0,0,0,0,9,0,1,0,1,GAB


In [40]:
##Promedio de medallas totales ganadas
df['Combined total'].mean()

120.4041095890411

In [41]:
df.sort_values(by=['Combined total'], ascending=False).head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
United States,26,976,757,666,2399,22,96,102,84,282,48,1072,859,750,2681,USA
Soviet Union,9,395,319,296,1010,9,78,57,59,194,18,473,376,355,1204,URS
Great Britain,27,236,272,272,780,22,10,4,12,26,49,246,276,284,806,GBR
Germany,15,174,182,217,573,11,78,78,53,209,26,252,260,270,782,GER
France,27,202,223,246,671,22,31,31,47,109,49,233,254,293,780,FRA


In [42]:
df.loc['Russia'] + df.loc['Soviet Union']

# Summer              14
Gold                 527
Silver               440
Bronze               438
Total               1405
# Winter              15
Gold.1               127
Silver.1              97
Bronze.1              94
Total.1              318
# Games               29
Gold.2               654
Silver.2             537
Bronze.2             532
Combined total      1723
ID                RUSURS
dtype: object

In [43]:
df.loc['Russia']

# Summer            5
Gold              132
Silver            121
Bronze            142
Total             395
# Winter            6
Gold.1             49
Silver.1           40
Bronze.1           35
Total.1           124
# Games            11
Gold.2            181
Silver.2          161
Bronze.2          177
Combined total    519
ID                RUS
Name: Russia, dtype: object

In [44]:
df.loc['Soviet Union']

# Summer             9
Gold               395
Silver             319
Bronze             296
Total             1010
# Winter             9
Gold.1              78
Silver.1            57
Bronze.1            59
Total.1            194
# Games             18
Gold.2             473
Silver.2           376
Bronze.2           355
Combined total    1204
ID                 URS
Name: Soviet Union, dtype: object

In [184]:
census_df

,SUMLEV,REGION,DIVISION,STATE,COUNTY,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
STNAME,,,,,,,,,,,,,,,,,,,,,
Alabama,40,3,6,1,0,Alabama,4779736,4780127,4785161,4801108,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
Alabama,50,3,6,1,1,Autauga County,54571,54571,54660,55253,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
Alabama,50,3,6,1,3,Baldwin County,182265,182265,183193,186659,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
Alabama,50,3,6,1,5,Barbour County,27457,27457,27341,27226,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
Alabama,50,3,6,1,7,Bibb County,22915,22919,22861,22733,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861
Alabama,50,3,6,1,9,Blount County,57322,57322,57373,57711,...,1.807375,-1.177622,-1.748766,-2.062535,-1.369970,1.859511,-0.848580,-1.402476,-1.577232,-0.884411
Alabama,50,3,6,1,11,Bullock County,10914,10915,10887,10629,...,-30.953709,-5.180127,-1.130263,14.354290,-16.167247,-29.001673,-2.825524,1.507017,17.243790,-13.193961
Alabama,50,3,6,1,13,Butler County,20947,20946,20944,20673,...,-14.032727,-11.684234,-5.655413,1.085428,-6.529805,-13.936612,-11.586865,-5.557058,1.184103,-6.430868
Alabama,50,3,6,1,15,Calhoun County,118572,118586,118437,117768,...,-6.155670,-4.611706,-5.524649,-4.463211,-3.376322,-5.791579,-4.092677,-5.062836,-3.912834,-2.806406


In [58]:
len(census_df['STNAME'].unique())

51

In [96]:
census_df[census_df['STNAME'] == 'Virginia']

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
STNAME,,,,,,,,,,,,,,,,,,,,,
Virginia,40,3,5,51,0,Virginia,Virginia,8001024,8001045,8025787,...,1.360760,0.614199,0.346268,-2.197521,-2.849964,5.004781,4.923775,4.447050,2.315622,1.793779
Virginia,50,3,5,51,1,Virginia,Accomack County,33164,33164,33165,...,1.625233,-2.312278,-9.650326,-0.090905,-1.606767,3.280563,-0.030030,-7.659947,2.212021,0.848858
Virginia,50,3,5,51,3,Virginia,Albemarle County,98970,98998,99236,...,5.393263,6.682955,2.841173,6.466120,6.426919,8.415091,9.960268,6.268154,10.384396,10.330677
Virginia,50,3,5,51,5,Virginia,Alleghany County,16250,16261,16211,...,6.267089,-3.008627,2.898729,-10.910565,-5.386395,6.451415,-2.947226,3.022080,-10.785872,-5.259656
Virginia,50,3,5,51,7,Virginia,Amelia County,12690,12695,12742,...,-2.117730,-1.098168,-1.729696,4.473571,9.576456,-0.392172,-0.549084,-1.179338,5.101440,10.199315
Virginia,50,3,5,51,9,Virginia,Amherst County,32353,32354,32385,...,-7.658440,10.655433,-6.280260,-3.423592,-3.094911,-7.596428,10.841284,-6.063700,-3.205727,-2.844817
Virginia,50,3,5,51,11,Virginia,Appomattox County,14973,14975,15035,...,-0.998735,6.171201,2.635133,2.623983,7.561930,-0.998735,6.171201,2.635133,2.623983,7.561930
Virginia,50,3,5,51,13,Virginia,Arlington County,207627,207676,209429,...,13.160459,3.500022,-1.779204,-17.263795,-8.464462,22.846518,13.447932,8.216526,-6.129001,2.720563
Virginia,50,3,5,51,15,Virginia,Augusta County,73750,73736,73478,...,2.294279,-1.584743,3.528338,1.551004,3.259852,2.579366,-1.246123,3.906857,1.996075,3.704377


In [111]:
##Contar cuántos County tiene un estado en particular
census_df[census_df['STNAME'] == max(census_df.loc[:]['COUNTY'])]

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
STNAME,,,,,,,,,,,,,,,,,,,,,


In [107]:
census_df.loc['Virginia', ['COUNTY', 'CTYNAME']].count()

COUNTY     134
CTYNAME    134
dtype: int64

In [121]:
len(census_df.loc['Virginia']['COUNTY'])

134

In [131]:
##STNAME con más COUNTY


In [166]:
census_df = census_df.set_index(['STNAME'])

In [134]:
len(census_df.loc['Virginia'])

134

In [175]:
county_count = pd.DataFrame()

In [174]:
census_df.loc['Virginia']['CTYNAME'].count()

134

In [188]:
state_df = pd.DataFrame(census_df['STNAME'].unique())

In [194]:
state_df[0]

0                  Alabama
1                   Alaska
2                  Arizona
3                 Arkansas
4               California
5                 Colorado
6              Connecticut
7                 Delaware
8     District of Columbia
9                  Florida
10                 Georgia
11                  Hawaii
12                   Idaho
13                Illinois
14                 Indiana
15                    Iowa
16                  Kansas
17                Kentucky
18               Louisiana
19                   Maine
20                Maryland
21           Massachusetts
22                Michigan
23               Minnesota
24             Mississippi
25                Missouri
26                 Montana
27                Nebraska
28                  Nevada
29           New Hampshire
30              New Jersey
31              New Mexico
32                New York
33          North Carolina
34            North Dakota
35                    Ohio
36                Oklahoma
3

In [195]:
for state in state_df:
    ctycount = census_df[state].count()
    state_df['COUNTIES'] = ctycount

KeyError: 0